In [3]:
import numpy as np

In [2]:
import pandas as pd
import functools
import math
import fractions


In [8]:
#Замена вектора на вектор из целых
def integer_collinear_vector(numbers, max_denominator=1000000):
    t = np.array([])
    for i in range(len(numbers)):
        if numbers[i] != 0:
            a = np.array([numbers[i]])
            t = np.concatenate((t, a))
    norma = np.min(t)
    numbers = [num / norma for num in numbers]
    fracts = [fractions.Fraction(f).limit_denominator(max_denominator) for f in numbers]
    denoms = [f.denominator for f in fracts]
    denoms_gcd = functools.reduce(math.gcd, denoms)
    denoms_lcm = functools.reduce(lambda a, b: a*b, [d // denoms_gcd for d in denoms]) * denoms_gcd
    result_integers = [f.numerator * (denoms_lcm // f.denominator) for f in fracts]
    return result_integers

Метод Якоби

In [12]:
def yakob(A,X,F,N):
    Y = np.zeros(N)
    for i in range(N):
        for j in range(N):
            if j == i:
                continue
            Y[i] += -(X[j]*(A[i][j]))
        Y[i] += F[i]
        Y[i] = Y[i]/(A[i][i])
    X = Y
    return X

In [13]:
#Размер матриц NxN
N = 2
A = np.array([[2,1],[1,2]])
F = np.array([1,-1])
X = np.zeros(N)
for i in range(10):
    X = yakob(A,X,F,N)
    print(X)

[ 0.5 -0.5]
[ 0.75 -0.75]
[ 0.875 -0.875]
[ 0.9375 -0.9375]
[ 0.96875 -0.96875]
[ 0.984375 -0.984375]
[ 0.9921875 -0.9921875]
[ 0.99609375 -0.99609375]
[ 0.99804688 -0.99804688]
[ 0.99902344 -0.99902344]


Метод Зейделя

In [8]:
def zeid(A,X,F,N):
    for i in range(N):
        X[i] = 0
        for j in range(N):
            if j == i:
                continue
            X[i] += -(X[j]*(A[i][j]))
        X[i] += F[i]
        X[i] = X[i]/(A[i][i])
    return X

In [10]:
#Размер матриц NxN
N = 2
A = np.array([[2,1],[1,2]])
F = np.array([1,-1])
X = np.zeros(N)
for i in range(10):
    X = zeid(A,X,F,N)
    print(X)

[ 0.5  -0.75]
[ 0.875  -0.9375]
[ 0.96875  -0.984375]
[ 0.9921875  -0.99609375]
[ 0.99804688 -0.99902344]
[ 0.99951172 -0.99975586]
[ 0.99987793 -0.99993896]
[ 0.99996948 -0.99998474]
[ 0.99999237 -0.99999619]
[ 0.99999809 -0.99999905]


Метод верхрней релаксации
<br>
$1<w<2$
<br>
$U_i^{s+1} =  U_i^s (1-w)  - \frac{w}{a_{ii}} (\sum_{j=1}^{i-1}a_{ij} u_j^{s+1} +\sum_{j=i+1}^{n}a_{ij} u_j^{s} - f_i)$

In [14]:
def relax(A,X,F,N,w):
    for i in range(N):
        X[i] = X[i]*(1-w)*A[i][i]/w
        for j in range(N):
            if j == i:
                continue
            X[i] += -(X[j]*(A[i][j]))
        X[i] += F[i]
        X[i] = X[i]*w/(A[i][i])
    return X

In [15]:
#Размер матриц NxN
N = 2
A = np.array([[2,1],[1,2]])
F = np.array([1,-1])
X = np.zeros(N)
w = 1.1
for i in range(20):
    X = relax(A,X,F,N,w)
    print(X)

[ 0.55   -0.8525]
[ 0.963875   -0.99488125]
[ 1.00079719 -1.00095033]
[ 1.00044296 -1.0001486 ]
[ 1.00003743 -1.00000573]
[ 0.99999941 -0.9999991 ]
[ 0.99999956 -0.99999985]
[ 0.99999996 -0.99999999]
[ 1. -1.]
[ 1. -1.]
[ 1. -1.]
[ 1. -1.]
[ 1. -1.]
[ 1. -1.]
[ 1. -1.]
[ 1. -1.]
[ 1. -1.]
[ 1. -1.]
[ 1. -1.]
[ 1. -1.]


Метод простых итераций:

$u + \tau A u = \tau f + u$
<br>
$u^{s+1} = u^s  + \tau(f - A u^s)$
<br>
$r^s := f - A u^s$
<br>
$u^{s+1} = u^s  + \tau r^s$
<br>
$\delta^s = u^s - u^{*}$

In [30]:
def k(u,t,A,f):
    u = u+(f-A@u)*t
    return  u

Вычисление максимального по модулю собственного числа

$\lambda^{s+1} = \frac{(u^s,Au^s)}{(u^s,u^s)}$
<br>
$O(\frac{|\lambda_{beforemax}|}{|\lambda_{max}|})$

In [17]:
def scal(x,y):
    m = x@(y.transpose())
    return m

In [21]:
u = np.array([1, 0])
t = 0.5
A = np.array([[5.0,-2.0],[-2,2]])
f = np.array([4, 5,6])
for j in range(5):
    print("Шаг №:", j+1)
    print("Вектор:",u)
    print("(u,Au):",scal(u,A@u))
    print("(u,u):",scal(u,u))
    print("lambda",scal(u,A@u)/scal(u,u))
    u = A@u

Шаг №: 1
Вектор: [1 0]
(u,Au): 5.0
(u,u): 1
lambda 5.0
Шаг №: 2
Вектор: [ 5. -2.]
(u,Au): 173.0
(u,u): 29.0
lambda 5.9655172413793105
Шаг №: 3
Вектор: [ 29. -14.]
(u,Au): 6221.0
(u,u): 1037.0
lambda 5.999035679845709
Шаг №: 4
Вектор: [173. -86.]
(u,Au): 223949.0
(u,u): 37325.0
lambda 5.999973208305425
Шаг №: 5
Вектор: [1037. -518.]
(u,Au): 8062157.0
(u,u): 1343693.0
lambda 5.9999992557823845


$tg (2\alpha) = \frac{2a_{ij}}{a_{ii} - a{jj}}$

Вычисление решения при матрице $A_{(m,n)}$, $rk(A) = n$:
<br>
$Au = f$
<br>
$A^{T} A x = A^{T} f$
<br>
$(A^{T}A)_{(n,n)}$
<br>
Тоесть это уже система с квадратной матрицей, которую мы решать умеем

Невязка:
<br>
$F(u) = \sum_i(\sum_j a_{ij}u_j - f_i)^2$

In [22]:
def nevya(u,A,f,n):
    s = 0
    for i in range(n):
        k = 0
        for j in range(n):
            k += (A[i][j] * u[j] - f[i])
        s += k ** 2
    return s

In [25]:
n = 2
A = np.array([[1,1],[2,-1],[1,3],[3,1]])
f = np.array([3,0.2,7.0,5.0])
u = np.linalg.inv((A.transpose()@A))@A.transpose()@f
print("Вектор:",u)
print("Невязка:",nevya(u,A,f,n))

Вектор: [1.03741935 1.96774194]
Невязка: 9.054851196670137
